
ВАЖНО! Чтобы ваше решение сохранялось и вы могли его нам предоставить в качестве ответа, сохраните себе копию ноутбука (Файл -> Сохранить копию на Диск).

Затем разрешите доступ к ноутбуку по ссылке (Поделиться -> Все у кого есть ссылка)

Вводная информация:

Мы занимаемся автоматизацией управления личных кабинетов клиентов, торгующих на маркетплейсах. В заданиях представлено 5 таблиц, отражающих упрощённые варианты источников, с которыми мы работаем ежедневно для расчёта параметров, составления отчетов, дашбордов и тд. В рамках заданий Вам необходимо проанализировать эти данные и представить результаты по каждой задаче.

С помощью данной тетрадки вы сможете ответить на вопросы тестового задания,используя для это SQL запросы или Python, в зависимости от личных предпочтений.  

В случае, если Вы не владеете навыками работы с SQL или Python, задания можно выполнять используя эксель. В этом случае мы настаиваем на выполнении дополнительных задач.

Для работы Вам будет полезно ознакомиться со структурой базы данных и описанием источников

[Структура БД](https://drive.google.com/file/d/1LRUrIpcgB4Bcv1kpSp-Vv7Gu3r4wTHTy/view?usp=drive_link)

[Описание БД](https://docs.google.com/document/d/1ms9ln6aZDRhM884Ddqr4ZUPQ_sryZ7VMWlX3EpEsuOg/edit?usp=sharing)

# SQL

In [4]:
import gdown
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize' : (12, 5)}, font='verdana')
plt.style.use('dark_background')
import sqlite3 as sql3

Подгружаю базу

In [5]:
url='https://drive.google.com/file/d/1hAtTBGUnAQjp3yMIx-i1Qx3QJ3XFjZmv/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
gdown.download(url, 'test_bd_for_sql.db', quiet=False)
connection = sql3.connect('test_bd_for_sql.db')

Downloading...
From: https://drive.google.com/uc?id=1hAtTBGUnAQjp3yMIx-i1Qx3QJ3XFjZmv
To: c:\Users\mail\OneDrive\Documents\my_python\2_ad_hoc\testing_me\test_bd_for_sql.db
100%|██████████| 6.03M/6.03M [00:00<00:00, 18.2MB/s]


Вывожу список таблиц базы для ознакомления

In [6]:
sql_query = """
            SELECT *
            FROM sqlite_master
            WHERE type = 'table'
            """
cursor = connection.cursor()
cursor.execute(sql_query)
a = cursor.fetchall()
t = []
for i in range(12):
    try:
        t.append(a[i][4].split("\"")[1])
    except IndexError:
        continue
t

['Nomenclature',
 'ComissionBySubjectName',
 'OrdersByDate',
 'WarehouseList',
 'StorageCostByDay']

## Задания

В рамках задач мы принимаем, что прибыль (маржа) это цена продажи минус все расходы. Что считать расходами в данном примере, мы оставляем на Ваше усмотрение.

Продажами считаем заказы, которые не были отменены и имеют статус "Клиентский"

Статистика источников собрана за ноябрь 2024 г.

В рамках заданий Вам необходимо будет рассчитывать метрики по всей артикульной матрице. Рекомендуется сначала совместить все таблицы и сохранить результат в отдельную переменную, с которой вы будете работать в дальнейшем.

Необходимо выполнить анализ продаж, основная таблица OrdersByDate. Для формирования сводной таблицы на основе OrdersByDate необходимо исследовать данные. 

In [7]:
query = """        SELECT * FROM Nomenclature LIMIT 2        """
pd.read_sql_query(query, connection)

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice
0,9428193,0,229720501,YORK,Щетки для стекол,342.92
1,9347354,0,98675218,YORK,Щетки для стекол,260.56


In [8]:
query = """ SELECT COUNT(NmId), COUNT(DISTINCT NmId) FROM Nomenclature   """
pd.read_sql_query(query, connection)

,COUNT(NmId),COUNT(DISTINCT NmId)
0,108,47


NmId не уникален и выполняет фунцию сопоставления в комбинации с SizeName

In [9]:
query = """ SELECT DISTINCT NmId, SizeName FROM Nomenclature ORDER BY NmId """
pd.read_sql_query(query, connection)

,NmId,SizeName
0,1362534,0
1,1407438,0
2,1472848,0
3,1668975,0
4,1927403,0
...,...,...
103,9174701,42
104,9174701,38
105,9326663,0
106,9347354,0


Все комбинации NmId и SizeName уникальны.

In [10]:
query = """ SELECT * FROM  OrdersByDate WHERE SizeName IS NULL """
pd.read_sql_query(query, connection)

,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,3747525,2024-11-16T12:46:11,344.0,0,Клиентский,210284,None
1,3757368,2024-11-16T12:47:47,352.0,0,Клиентский,210284,None
2,2496844,2024-11-16T12:51:41,396.0,0,Клиентский,210284,None
3,5767819,2024-11-16T12:58:37,797.0,0,Клиентский,206348,None
4,5767819,2024-11-16T12:58:37,797.0,0,Клиентский,206348,None
...,...,...,...,...,...,...,...
2010,2496844,2024-11-30T23:47:25,327.0,0,Клиентский,206348,None
2011,7059036,2024-11-30T23:49:11,874.0,0,Клиентский,206348,None
2012,7059036,2024-11-30T23:49:11,874.0,0,Клиентский,206348,None
2013,2496844,2024-11-30T23:59:58,327.0,0,Клиентский,206348,None


Таблица OrdersByDate содержит 5675 записей. В 2015 записях в колонке SizeName нет значений - указан NULL

In [11]:
query = """ 
        SELECT * 
        FROM  Nomenclature as n
        LEFT JOIN 
         (  
        SELECT * 
        FROM  OrdersByDate
        WHERE SizeName IS NULL
        GROUP BY  NmId,SizeName    ) as t  
        ON n.NmId = t.NmId

        WHERE t.SizeName IS NULL AND t.date IS NOT NULL
        """
pd.read_sql_query(query, connection)

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,9428193,0,229720501,YORK,Щетки для стекол,342.92,9428193,2024-11-16T14:31:12,897.00,0,Клиентский,210284,None
1,9347354,0,98675218,YORK,Щетки для стекол,260.56,9347354,2024-11-16T20:24:09,758.00,0,Клиентский,120762,None
2,9326663,0,109595410,YORK,Щетки косметические,90.76,9326663,2024-11-16T15:55:39,301.00,0,Клиентский,206348,None
3,7156668,S,350075652,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
4,7156668,M,350075653,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
5,7156668,L,350075654,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
6,7059036,0,318908541,YORK,Салфетки для уборки,440.80,7059036,2024-11-22T03:15:17,873.00,0,Клиентский,206348,None
7,6032372,0,238666916,YORK,Швабры,820.48,6032372,2024-11-17T13:22:08,2386.00,0,Клиентский,210284,None
8,5767819,0,101177417,YORK,Щетки для уборки,252.65,5767819,2024-11-16T12:58:37,797.00,0,Клиентский,206348,None
9,5717244,0,98674253,YORK,Губки для посуды,49.56,5717244,2024-11-16T14:27:37,373.00,0,Клиентский,206348,None


Почти все NULL - это не указанный размер 0. При сборке таблицы NULL в размерах заменим на размер 0.

In [12]:
query = """  SELECT * FROM WarehouseList LIMIT 2 """
pd.read_sql_query(query, connection)

,Origid,Warehouse,WorkTime,NearCityName
0,218987,Алматы Атакент,24/7,Махачкала
1,204939,Астана,24/7,Петропавловск-Камчатский


In [13]:
query = """  SELECT COUNT(Origid), COUNT(DISTINCT Origid) FROM WarehouseList LIMIT 2 """
pd.read_sql_query(query, connection)

,COUNT(Origid),COUNT(DISTINCT Origid)
0,205,121


In [14]:
query = """  SELECT * FROM WarehouseList ORDER BY Origid DESC"""
pd.read_sql_query(query, connection)

,Origid,Warehouse,WorkTime,NearCityName
0,336442,Щербинка,24/7,Москва
1,333530,СЦ Симферополь Молодежненское,24/7,Симферополь
2,324108,Астана 2,24/7,Москва
3,321932,Чашниково,24/7,Москва
4,318294,СЦ Софьино,24/7,Москва
...,...,...,...,...
200,1193,Хабаровск,24/7,Хабаровск
201,686,Новосибирск,24/7,Новосибирск
202,686,Новосибирск,24/7,Новосибирск
203,507,Коледино,24/7,Москва


Часть записей в таблице WarehouseList задвоена. При сборке сводной таблицы уберем из таблицы WarehouseList дубли следующим образом

In [15]:
query = """  SELECT DISTINCT Origid, Warehouse FROM WarehouseList ORDER BY Origid DESC"""
pd.read_sql_query(query, connection)

,Origid,Warehouse
0,336442,Щербинка
1,333530,СЦ Симферополь Молодежненское
2,324108,Астана 2
3,321932,Чашниково
4,318294,СЦ Софьино
...,...,...
116,2737,Санкт-Петербург (Уткина Заводь)
117,1733,Екатеринбург - Испытателей 14г
118,1193,Хабаровск
119,686,Новосибирск


В таблице OrdersByDate есть продажи со склада warehouseID = 0. В таблице WarehouseList склада с таким Origid нет

In [16]:
query = """ SELECT * FROM OrdersByDate WHERE warehouseID = 0 AND isCancel = 0 AND orderType="Клиентский" """
pd.read_sql_query(query, connection)

,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,5134688,2024-11-02 21:22:14,636.0,0,Клиентский,0,0
1,5534114,2024-11-07 16:10:13,3890.0,0,Клиентский,0,0
2,5134688,2024-11-07 22:33:04,905.0,0,Клиентский,0,0
3,5134688,2024-11-08 00:59:33,905.0,0,Клиентский,0,0
4,3475731,2024-11-08 20:00:19,711.0,0,Клиентский,0,32
...,...,...,...,...,...,...,...
273,9326663,2024-11-30T14:13:07,301.0,0,Клиентский,0,None
274,9326663,2024-11-30T17:49:41,301.0,0,Клиентский,0,None
275,5717244,2024-11-30T21:26:42,258.0,0,Клиентский,0,None
276,9347354,2024-11-30T22:07:05,636.0,0,Клиентский,0,None


Отсутствие соответствия в таблице WarehouseList не позволит получить стоимость хранения для этих продаж. Решим вопрос заполнения отсутствующей стоимости хранения позже.

In [17]:
query = """  SELECT * FROM StorageCostByDay WHERE OfficeId < 0 ORDER BY OfficeId """
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
1,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,0.245385
2,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
3,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,0.245385
4,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,213459505,0.245385
...,...,...,...,...,...,...
947,2024-11-30,-1000033,Пальто,NELIY VINCERE,129045785,2.410800
948,2024-11-30,-1000033,Пальто,NELIY VINCERE,129067756,2.410800
949,2024-11-30,-1000033,Куртки,NELIY VINCERE,241060323,2.410800
950,2024-11-30,-1000033,Куртки,Riches,254332716,2.410800


В таблице StorageCostByDay есть склад с отрицательным id. Это не является проблемой, поскольку в таблице WarehouseList этого склада нет, продаж с него не было

In [18]:
query = """  SELECT * FROM StorageCostByDay WHERE WarehousePrice < 0 ORDER BY OfficeId """
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
1,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
2,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,213459505,-0.024539
3,2024-11-02,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
4,2024-11-02,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
...,...,...,...,...,...,...
16254,2024-11-30,324108,Футболки,FRANK VANYA,126151327,-0.005075
16255,2024-11-30,324108,Майки спортивные,Flavour knit Gymnastic,213459548,-0.008678
16256,2024-11-30,324108,Купальники гимнастические,Flavour knit Gymnastic,213459615,-0.008678
16257,2024-11-30,324108,Купальники гимнастические,Flavour knit Gymnastic,319883878,-0.017357


В таблице StorageCostByDay есть отрицательные значения стоимости хранения (колонка WarehousePrice). Значение отрицательного знака в этом поле неизвестно.

In [19]:
query = """     SELECT * FROM StorageCostByDay WHERE ChrtId = "31789515" ORDER BY OfficeId limit 2"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,507,Водолазки,Flavour knit,31789515,0.1015
1,2024-11-01,507,Водолазки,Flavour knit,31789515,0.5075


В таблице StorageCostByDay есть полные дубли записей с разной стоимостью хранения. Причины появления дублей и принцип определения для них значения WarehousePrice неизвестны.

Вывожу амплитуду разброса значений WarehousePrice для дублей полей

In [20]:
query = """     SELECT *, MIN(WarehousePrice), MAX(WarehousePrice), MAX(WarehousePrice) - MIN(WarehousePrice) as diff
                FROM StorageCostByDay
                GROUP BY Date, ChrtId, OfficeId, brand, Subject
                HAVING diff <> 0   
                Order by diff DESC
                   
"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice,MIN(WarehousePrice),MAX(WarehousePrice),diff
0,2024-11-22,206348,Щетки для уборки,YORK,98675540,234.57280,-11.728640,234.57280,246.301440
1,2024-11-23,206348,Щетки для уборки,YORK,98675540,229.74336,-11.487168,229.74336,241.230528
2,2024-11-24,206348,Щетки для уборки,YORK,98675540,220.08448,-11.004224,220.08448,231.088704
3,2024-11-25,206348,Щетки для уборки,YORK,98675540,206.97600,-10.348800,206.97600,217.324800
4,2024-11-26,206348,Щетки для уборки,YORK,98675540,200.76672,-10.038336,200.76672,210.805056
...,...,...,...,...,...,...,...,...,...
15034,2024-11-13,206348,Водолазки,Flavour knit,31789515,0.15400,0.077000,0.15400,0.077000
15035,2024-11-14,206348,Водолазки,Flavour knit,31789515,0.15400,0.077000,0.15400,0.077000
15036,2024-11-15,206348,Водолазки,Flavour knit,31789515,0.15400,0.077000,0.15400,0.077000
15037,2024-11-25,321932,Книги,Атлас принт,112850319,0.07000,-0.003500,0.07000,0.073500


Различие стоимости хранения существует для 15039 комбинаций даты, склада, товара, группы и размера. 

В условиях неопределенности выбираю консервативную стратегию защиты прибыли клиента и в качестве значения WarehousePrice далее выбираю максимальное значение.

In [21]:
query = """     SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                FROM StorageCostByDay
                GROUP BY ChrtId, Date, OfficeId, brand, Subject"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehouseCost
0,2024-11-19,117501,Костюмы,LAVIRA,24078191,2.975000
1,2024-11-20,117501,Костюмы,LAVIRA,24078191,2.975000
2,2024-11-06,2737,Платья,LAVIRA,25556765,4.704000
3,2024-11-07,2737,Платья,LAVIRA,25556765,4.704000
4,2024-11-08,2737,Платья,LAVIRA,25556765,4.704000
...,...,...,...,...,...,...
51404,2024-11-12,1733,Швабры,YORK,416723700,1.247344
51405,2024-11-13,1733,Швабры,YORK,416723700,1.247344
51406,2024-11-14,1733,Швабры,YORK,416723700,1.247344
51407,2024-11-15,1733,Швабры,YORK,416723700,1.247344


Собираю сводную таблицу, исключая записи, не считающиеся продажей (возвраты и не "клиентские" заказы)

In [22]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, ord.warehouseID,
                            --nom.BrandName, nom.SubjectName, nom.CostPrice, 
                            nom.ChrtId,
                            com.Commission,  com.CategoryName,
                            st.WarehouseCost
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
                                LEFT JOIN 
                                  (SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                                  FROM StorageCostByDay
                                  GROUP BY ChrtId, Date, OfficeId, brand, Subject
                                  ) as st
                                  ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
            WHERE             iscancel = 0 AND orderType = "Клиентский"
                              )
        
        SELECT              *
        FROM                cte
        WHERE              WarehouseCost IS NULL
      
        """
pd.read_sql_query(query, connection)

,date,NmId,SizeName,SoldPrice,warehouseID,ChrtId,Commission,CategoryName,WarehouseCost
0,2024-11-04,4558489,46-48,903.0,210284,120692596.0,24.5,Одежда,None
1,2024-11-05,4558489,54-56,1308.0,210284,120692598.0,24.5,Одежда,None
2,2024-11-06,4558489,54-56,1003.0,210284,120692598.0,24.5,Одежда,None
3,2024-11-07,4558489,42-44,928.0,210284,123737052.0,24.5,Одежда,None
4,2024-11-08,4558489,42-44,1003.0,210284,123737052.0,24.5,Одежда,None
...,...,...,...,...,...,...,...,...,...
2226,2024-11-30,9428193,0,870.0,210284,229720501.0,19.5,Хозяйственные товары,None
2227,2024-11-30,5717244,0,258.0,0,98674253.0,19.5,Хозяйственные товары,None
2228,2024-11-30,9347354,0,636.0,0,98675218.0,19.5,Хозяйственные товары,None
2229,2024-11-30,9428193,0,870.0,210284,229720501.0,19.5,Хозяйственные товары,None


Для многих комбинаций значений таблицы StorageCostByDay нет соответствий в таблице OrdersByDate: 2231 запись не имеет данных в колонке WarehouseCost

Стоимость хранения определяется в первую очередь весом и габаритами. В рамках консервативной стратегии определяю недостающие значения WarehouseCost для каждого артикула как максимальную стоимость хранения товарной группы (Subject), к которой принадлежит конкретный артикул

In [23]:
query = """     SELECT Subject, MAX(WarehousePrice) as max
                FROM StorageCostByDay
                GROUP BY Subject
                ORDER BY max DESC   
                   
"""
pd.read_sql_query(query, connection)

,Subject,max
0,Щетки для уборки,234.572800
1,Стулья,195.336960
2,Карты географические,126.772800
3,Салфетки для уборки,92.133888
4,Полотенца банные,79.296000
...,...,...
71,Кремы,0.101500
72,Джемперы,0.101500
73,Декор для маникюра,0.101500
74,Браслеты,0.101500


Добавляю недостающую стоимость хранения и собираю итоговую таблицу

In [25]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, nom.CostPrice, com.Commission,
                            IIF(WarehouseCost IS NULL, max.max, WarehouseCost) as WarehousePrice,
                            com.CategoryName, max.Subject,nom.BrandName
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
                                LEFT JOIN 
                                  (SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                                  FROM StorageCostByDay
                                  GROUP BY ChrtId, Date, OfficeId, brand, Subject
                                  ) as st
                                  ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
                                LEFT JOIN
                                  (SELECT Subject, MAX(WarehousePrice) as max
                                   FROM StorageCostByDay
                                   GROUP BY Subject) as max
                                  ON  max.Subject = nom.SubjectName
            WHERE             iscancel = 0 AND orderType = "Клиентский")
      
        SELECT              *
        FROM                cte
        ORDER BY            date, NmId, SizeName

        """
pd.read_sql_query(query, connection)

,date,NmId,SizeName,SoldPrice,CostPrice,Commission,WarehousePrice,CategoryName,Subject,BrandName
0,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
1,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
2,2024-11-01,1362534,0,513.12,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
3,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
4,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
...,...,...,...,...,...,...,...,...,...,...
5130,2024-11-30,9428193,0,870.00,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK
5131,2024-11-30,9428193,0,870.05,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK
5132,2024-11-30,9428193,0,870.05,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK
5133,2024-11-30,9428193,0,870.00,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK


Добавляю расчет полной себестоимости с учетом комиссии и стоимости хранения, а так же маржу и прибыль

In [26]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, nom.CostPrice,
                            ROUND(nom.CostPrice + ord.SoldPrice*com.Commission/100, 2) + IIF(WarehouseCost IS NULL, max.max, WarehouseCost) as cost_full,
                            com.CategoryName, max.Subject,nom.BrandName
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
                                LEFT JOIN 
                                  (SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                                  FROM StorageCostByDay
                                  GROUP BY ChrtId, Date, OfficeId, brand, Subject
                                  ) as st
                                  ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
                                LEFT JOIN
                                  (SELECT Subject, MAX(WarehousePrice) as max
                                   FROM StorageCostByDay
                                   GROUP BY Subject) as max
                                  ON  max.Subject = nom.SubjectName
            WHERE             iscancel = 0 AND orderType = "Клиентский")
      
        SELECT              *, SoldPrice - cost_full as profit, ROUND((SoldPrice - cost_full)*100/cost_full, 1) as margin
        FROM                cte
        ORDER BY            date, NmId, SizeName

        """
pd.read_sql_query(query, connection)
df = pd.read_sql_query (query, connection)

И еще один сводник сохраняю в отдельный датафрейм для dashborad-а.

In [29]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, nom.CostPrice,
                            ROUND(nom.CostPrice + ord.SoldPrice*com.Commission/100, 2) + IIF(WarehouseCost IS NULL, max.max, WarehouseCost) as cost_full,
                            com.CategoryName, max.Subject,nom.BrandName,
                            ord.iscancel
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
                                LEFT JOIN 
                                  (SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                                  FROM StorageCostByDay
                                  GROUP BY ChrtId, Date, OfficeId, brand, Subject
                                  ) as st
                                  ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
                                LEFT JOIN
                                  (SELECT Subject, MAX(WarehousePrice) as max
                                   FROM StorageCostByDay
                                   GROUP BY Subject) as max
                                  ON  max.Subject = nom.SubjectName)
      
        SELECT              *, SoldPrice - cost_full as profit, ROUND((SoldPrice - cost_full)*100/cost_full, 1) as margin
        FROM                cte
        ORDER BY            date, NmId, SizeName

        """
pd.read_sql_query(query, connection)
df_dash = pd.read_sql_query (query, connection)

In [30]:
df.head(1)

,date,NmId,SizeName,SoldPrice,CostPrice,cost_full,CategoryName,Subject,BrandName,profit,margin
0,2024-11-01,1362534,0,529.0,181.52,316.3196,Красота,Щетки косметические,YORK,212.6804,67.2


In [31]:
df = df.dropna()

In [32]:
df.columns = ['date', 'id', 'size', 'sold_price', 'cost_price','cost_full', 'category',
       'subject', 'brand', 'profit', 'margin']
df.head(1)

,date,id,size,sold_price,cost_price,cost_full,category,subject,brand,profit,margin
0,2024-11-01,1362534,0,529.0,181.52,316.3196,Красота,Щетки косметические,YORK,212.6804,67.2


In [33]:
df.date = pd.to_datetime(df.date)

### Задание 1
<b>Рассчитать показатели:</b>
```
- среднюю цену заказа каждого артикула
- скорость заказов
```
В ответе вывести топ 5 артикулов по скорости заказов

In [34]:
result = df\
            .groupby('id',as_index = False)\
            .agg(average_price=('sold_price', 'mean'), 
                 days=('date', lambda x: (x.max() - x.min()).days),
                 cnt=('cost_full', lambda x: len(x)))
result['speed'] = round(result.cnt / result.days, 1)
result['average_price']=result['average_price'].round(2)
result = result[['id','average_price','speed']]
result.sort_values('speed', ascending=False).head(5)

,id,average_price,speed
7,2496844,354.12,42.1
10,3176618,733.43,13.5
39,9326663,303.53,13.3
16,3757368,313.65,13.1
41,9428193,863.62,12.0


### Задание 2
<b>Рассчитать показатели на основе продаж:</b>
```
- прибыль (маржа) за месяц, руб.
- рентабельность (маржинальность), %
- медианную прибыль (маржу) за месяц, руб.
- оборачиваемость (дн)
```
В ответе вывести топ 5 артикулов по прибыли в руб

In [35]:
result2 = df\
            .groupby('id',as_index = False)\
            .agg(revenue=('sold_price', 'sum'), 
                 costs=('cost_full', 'sum'),
                 turnover=('id', lambda x: len(x)/30),
                 median_margin=('margin','median')                 )
result2['profit']=round((result2.revenue-result2.costs),1)
result2['turnover']=round(result2.turnover,1)
result2['margin']=round((result2.revenue-result2.costs)*100/result2.costs, 1)
result2=result2[['id', 'profit', 'margin', 'median_margin', 'turnover']]
result2.sort_values('profit', ascending = False).head(5)


,id,profit,margin,median_margin,turnover
5,2252229,217745.2,61.9,55.4,1.8
4,2121671,184462.1,68.1,73.5,1.4
7,2496844,151119.5,53.7,48.1,40.7
10,3176618,114896.5,66.8,71.9,13.0
41,9428193,113140.5,60.1,60.3,11.6


Задание 3
### Задание 3
<b>Рассчитать показатели:</b>
```
- Стоимость хранения за месяц
- Средняя стоимость хранения в день
```
В ответе вывести топ 5 артикулов по средней стоимости хранения

In [36]:
result3 = df\
            .groupby('id',as_index = False)\
            .agg(cost_price_month=('cost_price', 'sum'))  
result3.cost_price_month = result3.cost_price_month.round(1)
result3['cost_price_per_day']= round(result3.cost_price_month/30,1)
result3.sort_values('cost_price_per_day', ascending=False).head(5)

,id,cost_price_month,cost_price_per_day
5,2252229,248391.0,8279.7
4,2121671,189278.6,6309.3
7,2496844,129886.4,4329.5
41,9428193,119679.1,3989.3
2,1472848,112341.7,3744.7



### Дополнительное задание 1
<b>Анализ данных</b>
```
Проведите анализ получившейся статистики по кабинету.
Какие решения можно принять по работе с ассортиментом?
Учесть, что поставок не будет ближайшие 30 дней, а отсутствие остатка влечет к снижению рейтинга карточки товара.
Какие выводы можно сделать по брендам на онове всей статистики за ноябрь?
```


In [37]:
df.head(1)

,date,id,size,sold_price,cost_price,cost_full,category,subject,brand,profit,margin
0,2024-11-01,1362534,0,529.0,181.52,316.3196,Красота,Щетки косметические,YORK,212.6804,67.2


Данные позволяют проанализировать

- Оборот (сумма продаж)
- Прибыль (доходы минус расходы)
- Маржу (отношение прибыли к себестоимости)
- Количество проданного товара в штуках
- Затраты хранение товара
- Комиссию WB

Это можно сделать в разрезах:

- Номенклатуры (артикулы)
- Товарной категории (category)
- Товарной группы (subject)
- Торговой марки (brand)

На основании этих данных создам дашборд, например, в google sheets, где можно компактно и наглядно визуализировать результаты анализа

Так же можно рассмотреть дополнительные аналитические разрезы, через добавление категориальных переменных, например:

- продажи с прибылью или с убытком
- каждый день продается товар или нет
- разделить амплитуду маржинальность на сегменты: высокая, средняя и низкая маржа и сегментировать по ним номенклатуру/бренды/группы
- и так далее

Подготовлю так же основу для многофакторного анализа (отбор товара под нескольким параметрам одновременно). Например, товар с максимальной доходностью может иметь очень маленькие продажи и отбор по доходности не является эффективным.

Проведу сегментацию номенклатуры по следующему набору параметров: продажи, прибыль, проданное количество

In [38]:
df_test = df.groupby('id',as_index = False).agg({'sold_price'  :  'sum',
                                'margin'      :  'mean',
                                'cost_full'   :  'count'})
df_test.columns = ['id', 'revenue', 'margin', 'cnt']
df_test.head(1)

,id,revenue,margin,cnt
0,1362534,123068.39,66.762613,222


Категоризирую данные параметров через попадание значений в диапазоны, заданные квантилями

In [39]:
quantiles = df_test[['revenue','margin','cnt']].quantile(q=[0.25,0.5,0.75])
quantiles

,revenue,margin,cnt
0.25,14827.605,53.956665,17.50
0.50,49535.000,66.982526,51.50
0.75,142898.050,81.302060,135.75


Готовлю функцию для назначения рейтинга в зависимости от попадания в тот или иной квантиль

Принцип назначения: 1 - лучший, 4 - худший

In [40]:
def categorize_data (value, metric):
    if value <= quantiles[metric][0.25]:
        return 4
    elif value <= quantiles[metric][0.5]:
        return 3
    elif value <= quantiles[metric][0.75]:
        return 2
    else:
        return 1

Применяю функцию к нужным полям

In [41]:
df_test['Revenue']= df_test['revenue'].apply(lambda x: categorize_data(x, 'revenue'))
df_test['Margin']= df_test['margin'].apply(lambda x: categorize_data(x, 'margin'))
df_test['Cnt']= df_test['cnt'].apply(lambda x: categorize_data(x, 'cnt'))


In [42]:
df_test= df_test[['id', 'Revenue', 'Margin', 'Cnt']]
df_test.head()

,id,Revenue,Margin,Cnt
0,1362534,2,3,1
1,1407438,2,3,2
2,1472848,1,4,2
3,1668975,4,2,4
4,2121671,1,2,3


Собираю рейтинги в одно поле

In [43]:
df_test['rmc_score'] = df_test['Revenue'].astype(str) + df_test['Margin'].astype(str) + df_test['Cnt'].astype(str)
df_test = df_test[['id', 'rmc_score']]
df_test.sort_values('rmc_score', ascending = True)

,id,rmc_score
10,3176618,121
40,9347354,121
27,5534114,122
4,2121671,123
41,9428193,131
34,7167361,131
5,2252229,132
32,7059036,141
7,2496844,141
29,5767819,141


Каждый артикул имеет рейтинг, включающий 3 параметра.

Идеальный товар 111: он дорогой, с большой маржой и хорошо продается. В данных нет таких товаров, однако есть 121. Товар 121  дорогой, маржа выше среднего и хорошо продается. Так же есть товары 211: дороже среднего, маржа большая и хорошо продается. И так далее.

Аналогично определяются аутсайдеры. Например, 434 - это артикулы, которым свойственны дешевые, низкомаржинальные и редкие продажи.


Рейтинг 

In [44]:
df_test.rmc_score.value_counts().sort_values(ascending = True).head(5)

rmc_score
213    1
241    1
123    1
132    1
231    1
Name: count, dtype: int64

Лучшие товары компании

In [45]:
df_test[(df_test.rmc_score == '121') | (df_test.rmc_score == '211') ] 

,id,rmc_score
10,3176618,121
16,3757368,211
40,9347354,121


Антирейтинг

In [46]:
df_test.rmc_score.value_counts().sort_values(ascending = False).head(5)

rmc_score
424    4
434    4
141    3
121    2
413    2
Name: count, dtype: int64

In [47]:
df_test[(df_test.rmc_score == '424') | (df_test.rmc_score == '434')] 

,id,rmc_score
3,1668975,424
6,2273131,434
13,3475731,434
22,5085510,424
24,5145887,424
35,7970311,434
36,8190735,424
38,8993028,434


Артикулы, которые ухудшают статистику продаж.

In [48]:
df_test[(df_test.rmc_score == '424') | (df_test.rmc_score == '434')] 

,id,rmc_score
3,1668975,424
6,2273131,434
13,3475731,434
22,5085510,424
24,5145887,424
35,7970311,434
36,8190735,424
38,8993028,434


Проведем такой же анализ в разрезе торговых марок

In [49]:
brands = df.groupby('brand',as_index = False).agg({'sold_price'  :  'sum',
                                'margin'      :  'mean',
                                'cost_full'   :  'count'})
brands.columns = ['id', 'revenue', 'margin', 'cnt']
brands

,id,revenue,margin,cnt
0,Azur,18481.00,80.300000,20
1,Flavour knit,17354.24,91.835294,17
2,Flavour knit Gymnastic,99129.20,89.480645,62
3,YORK,2657077.10,62.106190,4475
4,Авангард,230100.04,78.165753,73
5,Атлас принт,269691.93,68.477419,372
6,Экомебель,1025145.00,64.066667,96


In [50]:
quantiles = brands[['revenue','margin','cnt']].quantile(q=[0.25,0.5,0.75])
quantiles

,revenue,margin,cnt
0.25,58805.100,66.272043,41.0
0.50,230100.040,78.165753,73.0
0.75,647418.465,84.890323,234.0


In [51]:
brands['Revenue']= brands['revenue'].apply(lambda x: categorize_data(x, 'revenue'))
brands['Margin']= brands['margin'].apply(lambda x: categorize_data(x, 'margin'))
brands['Cnt']= brands['cnt'].apply(lambda x: categorize_data(x, 'cnt'))

In [52]:
brands= brands[['id', 'Revenue', 'Margin', 'Cnt']].copy()
brands.head()

,id,Revenue,Margin,Cnt
0,Azur,4,2,4
1,Flavour knit,4,1,4
2,Flavour knit Gymnastic,3,1,3
3,YORK,1,4,1
4,Авангард,3,3,3


In [53]:
brands['rmc_score'] = brands['Revenue'].astype(str) + brands['Margin'].astype(str) + brands['Cnt'].astype(str)
brands = brands[['id', 'rmc_score']]
brands.sort_values('rmc_score', ascending = True)

,id,rmc_score
3,YORK,141
6,Экомебель,142
5,Атлас принт,231
2,Flavour knit Gymnastic,313
4,Авангард,333
1,Flavour knit,414
0,Azur,424


Очевидные аутсайдеры в брендах Flavour и Azur - несмотря на хорошую маржу они дешевые и плохо продаются. Источниками прибыли при этом являются YORK и Экомебель

In [55]:
df_dash.to_excel('dash.xlsx', index = False)

Дашборд: https://docs.google.com/spreadsheets/d/1I3q61xf5IHLnWiZgF9hV5SUMgyWjoNy3GarqY_KieJ0/edit?usp=drive_link